In [1]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import os
import csv
import textract
import nltk
import joblib

import MeCab
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from konlpy.tag import *
from collections import Counter
from nltk.corpus import stopwords
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline

In [2]:
df = pd.read_csv('adoor_data/sns_feed.csv', encoding='UTF8')
df['created_at'] = pd.to_datetime(df['created_at'])

start_date = pd.Timestamp(2019, 1, 28, 0)
end_date = pd.Timestamp(2019, 4, 1, 0)

mask = (df['created_at'] > start_date) & (df['created_at'] <= end_date)
df = df.loc[mask]

df.head()

,adoor,SNS,SNS 아이디,private,created_at,photo,content
0,2,facebook,김유리,False,2019-02-08,홍보,저와 팀원들이 영혼을 갈아넣은 서비스 베타버전이 드디어 나왔어요!!🤩\n밤잠을 줄여...
1,2,instagram,nnnyu_ri,False,2019-02-08,홍보,https://adoor.app @adoor.team\n저와 팀원들이 영혼을 갈아넣...
2,2,instagram,yulo_mon,True,2019-02-07,"가족, 아기, 강아지",카조꾸또 잇쇼니 토떼모 우레시깟딴데쓰
3,2,instagram,yulo_mon,True,2019-02-25,일상,🙋🏻‍♀️
4,2,instagram,yulo_mon,True,2019-03-02,나들이,필름캐머러 첫 롤 사진들1\n수라미가 준 로모그래피 심플유즈 기본 장착 컬러네거티브...


In [3]:
def getNVM_lemma(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    pos = []
    tags = ['NNG','NNP','VV','VA', 'VX', 'VCP','VCN']
    for word_ in word_tag[:-2]:
        word = word_.split("\t")
        tag = word[1].split(",")
        if(len(word[0]) < 2) or ("게" in word[0]):
            continue
        if(tag[-1] != '*'):
            t = tag[-1].split('/')
            if(len(t[0]) > 1 and ('VV' in t[1] or 'VA' in t[1] or 'VX' in t[1])):
                pos.append(t[0])
        else:
            if(tag[0] in tags):
                pos.append(word[0])
    return pos

In [4]:
tf_vect = CountVectorizer(tokenizer=getNVM_lemma, min_df=2, max_df=6000, max_features=25000)
dtm = tf_vect.fit_transform(df['content'].values.astype('U'))

n_topics = 4

lda = LatentDirichletAllocation(n_components=n_topics, topic_word_prior=0.01, doc_topic_prior=0.001)
lda.fit(dtm)
saved_model = joblib.dump(dtm, 'LDA_IP.pkl')

In [5]:
names = tf_vect.get_feature_names()
topics_word = dict()
n_words = 10

for idx, topic in enumerate(lda.components_):
    vocab = []
    for i in topic.argsort()[:-(n_words-1):-1]:
        vocab.append((names[i], topic[i].round(2)))
    topics_word[idx+1] = [(names[i], topic[i].round(2)) for i in topic.argsort()[:-(n_words-1):-1]]
max_dict = dict()
for idx, vec in enumerate(lda.transform(dtm)):
    t = vec.argmax()
    if(t not in max_dict):
        max_dict[t] = (vec[t], idx)
    else:
        if(max_dict[t][0] < vec[t]):
            max_dict[t] = (vec[t], idx)
            
sorted_review = sorted(max_dict.items(), key = lambda x: x[0], reverse=False)

for key, value in sorted_review:
    print('주제 {}: {}'.format(key+1, topics_word[key+1]))
    print('[주제 {}의 대표 리뷰 :{}]\n{}\n\n'.format(key+1, value[0], df['content'].values.astype('U')[value[1]]))

주제 1: [('사람', 12.99), ('사용', 6.01), ('친구', 4.06), ('후회', 4.01), ('가입', 4.01), ('만들', 3.55), ('그러', 3.01), ('떠나', 3.01)]
[주제 1의 대표 리뷰 :0.999857170062845]
창진이가 정자까지 저녁을 먹으러 왔다! 그 사람을 얼마나 좋아하느냐는 그 사람을 위해 얼만큼의 귀찮음까지 무릅쓸 수 있느냐고 하잖나. 저녁 약속 하나로 만나러 오긴 꽤 먼 거리인데 귀찮음을 무릅쓰고 얼굴 보러 와주는 사람들이 많아 참 감사하다 요즘. 날 좋아하는 사람이 참 많고만! 헤헤 🙈

내가 좋아하고 아끼는 많은 사람들이 내일 학사모를 쓴다. 떠나는 사람보다 남아있는 사람이 더 빈자리를 크게 느끼고 그리워한다고 그랬다. 내가 떠나는 사람일 땐 몰랐는데, 남은 사람이 되니 벌써부터 헛헛하다.

관악을 떠나도, 지금껏 그랬던 것처럼 귀찮음 따위 아주 쉽게 무릅쓰고 만나 우리!


주제 2: [('사진', 6.01), ('생각', 6.01), ('사람', 6.01), ('머리', 4.01), ('동안', 4.01), ('통제', 3.01), ('느끼', 3.01), ('의미', 3.01)]
[주제 2의 대표 리뷰 :0.9998928724467931]
그냥 뻘글_안 읽는 거 추천
.
.
.
.
.
.
.
.
.
.
.

작년 이맘때쯤 영 좋지 않은 식습관이 나의 삶과 일과의 주축이 된 적이 있었는데 문득 그때 생각이 났음. 몸과 마음 둘 다 상해도 그 누구에게도 말 못 하고 혼자 버티려고 노력했었음. 난 당시 그 식습관을 x라 불렀는데 나날이 x의 제패의 연속이었음. X는 나를 매일 아침 6시에 일어나게 했고 조울증 약을 먹게 했고 내 모든 행동을 감시하고 통제했음. 밤에 잠 못 들게 했고 또한 하루 종일 자게 했음. 밖에서 쓰러졌을 때 혹 링거 맞으면 살찔까 두려워 병원도 못 가게 했고 마찬가지의 이유로 감기약도 못 먹게 했음. 1년 동안 생리를 못했고 수십번 기절하면서 자연스레 머리로 먼저

In [6]:
visual = pyLDAvis.sklearn.prepare(lda_model=lda, dtm=dtm, vectorizer=tf_vect)
pyLDAvis.save_html(visual, 'LDA_Visualization2.html')
pyLDAvis.display(visual)